# Imports

In [1]:
# System
import os 
import sys
import re
import json
from pathlib import Path
from dotenv import load_dotenv

# Data Type
import json
from textwrap import dedent
from pprint import pprint

# Configs

In [2]:
# Add project root to path
sys.path.append(str(Path.cwd().parent))
print("Project root added to path.")

env_path = Path.cwd().parent / '.env'
load_dotenv(dotenv_path=env_path)

# Load environment variables
print(f"Environment variables loaded: {load_dotenv()}.")

Project root added to path.
Environment variables loaded: True.


# Gemini Model

In [3]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI


model = 'gemini-2.5-flash-preview-04-17'

llm = ChatGoogleGenerativeAI(
    api_key=os.environ.get("GEMINI_API_KEY"),
    model=model, 
    temperature=0.6,
)
llm.invoke("Hello, testing connections!")

AIMessage(content="Hello! Connection confirmed. 👋\n\nEverything looks good on my end. I'm here and ready to help whenever you are!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--6ecea5dc-e6c8-4f11-a7f3-ff38157c80fe-0', usage_metadata={'input_tokens': 6, 'output_tokens': 26, 'total_tokens': 362, 'input_token_details': {'cache_read': 0}})

# PGVector

In [11]:
from langchain_postgres.vectorstores import PGVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-exp-03-07",
    google_api_key=os.environ.get("GEMINI_API_KEY")
)

# See docker command above to launch a postgres instance with pgvector enabled.
pg_user = os.environ.get('DB_USER')
pg_password = os.environ.get('DB_PASSWORD')
pg_db = os.environ.get('DB_NAME')
pg_host = os.environ.get('DB_HOST')
pg_port = os.environ.get('DB_PORT')
schema = 'vector_store,public'
connection = (
    f"postgresql+psycopg://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
    f"?options=-csearch_path%3D{schema}"
)
collection_name = "game_embeddings"
distance_strategy = 'cosine'

vector_store = PGVector(
    embeddings=embeddings,
    collection_name=collection_name,
    connection=connection,
    use_jsonb=True,
    distance_strategy=distance_strategy
)

vector_store.add_texts(
    texts=[
        "hello",
        "world",
    ],
    metadatas=[
        {"source": "hello"},
        {"source": "world"},
    ]
)

['3638bb2a-99ec-4aa1-a2de-93c5bf60f5ee',
 'f24b21d6-93ec-4850-9add-6dc49d5c0ca6']

In [ ]:
from langchain_core.documents import Document

docs = []

for _, row in df.iterrows():
    text = f"{row['title']}\n\n{row['body']}"
    metadata = {
        "id": row["id"],
        "category": row["category"],
        "published_at": str(row["published_at"])
    }
    docs.append(Document(page_content=text, metadata=metadata))
    

vector_store.add_documents(docs)